In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.layers import Input, Dense, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
import optuna

# Load and preprocess the data
data = pd.read_csv('/Users/Hsiao-ping.ni/ASU Dropbox/Hsiao-Ping Ni/PhD/Data/4th paper- Chiller/Diffusion_Anomaly_Detection_Results.csv', encoding='unicode_escape')
#data = pd.read_csv('/Users/hsiaopingni/Desktop/Hsiao-Ping PhD/Data/paper- Chiller/Diffusion_Anomaly_Detection_Results.csv', encoding='unicode_escape')
data = data[['ï»¿Discharge Temp (F)', 'Input % full load amps (Motor) (%)', 'Condenser liq temp IN (F)',
             'Condenser liq temp OUT (F)', 'Chilled liq temp IN (F)', 'Chilled liq temp OUT (F)',
             'Condenser saturation (F)', 'Evaporator saturation (F)', 'Evaporator pressure (PSIG)',
             'Condenser pressure (PSIG)', 'Oil sump temp (F)', 'Oil pressure (PSIG)', 'Anomaly']]

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.drop(columns=['Anomaly']))  # Normalize only the features

# Separate anomalies and normal data
anomalies = data[data['Anomaly'] == 1]  # Assuming '1' is the label for anomalies
normal_data = data[data['Anomaly'] == 0]  # Assuming '0' is the label for normal data

# Normalize the data separately for anomalies and normal data
anomalies_scaled = scaler.transform(anomalies.drop(columns=['Anomaly']))
normal_data_scaled = scaler.transform(normal_data.drop(columns=['Anomaly']))

# Define the Sampling layer for VAE
class Sampling(Layer):
    def call(self, inputs):
        mean, log_var = inputs
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mean + tf.exp(0.5 * log_var) * epsilon

# Define VAE architecture
def build_vae(input_dim, hidden_dims, latent_dim):
    # Encoder
    inputs = Input(shape=(input_dim,))
    x = inputs
    for dim in hidden_dims:
        x = Dense(dim, activation='relu')(x)
    mean = Dense(latent_dim)(x)
    log_var = Dense(latent_dim)(x)
    z = Sampling()([mean, log_var])
    encoder = Model(inputs, [mean, log_var, z], name="encoder")
    
    # Decoder
    latent_inputs = Input(shape=(latent_dim,))
    x = latent_inputs
    for dim in reversed(hidden_dims):
        x = Dense(dim, activation='relu')(x)
    outputs = Dense(input_dim, activation='sigmoid')(x)
    decoder = Model(latent_inputs, outputs, name="decoder")
    
    # VAE Model
    reconstructed = decoder(encoder(inputs)[2])
    vae = Model(inputs, reconstructed, name="vae")
    
    # VAE Loss
    reconstruction_loss = mse(inputs, reconstructed) * input_dim
    kl_loss = 1 + log_var - tf.square(mean) - tf.exp(log_var)
    kl_loss = tf.reduce_sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    
    return vae, encoder, decoder

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters using TPE
    hidden_dim1 = trial.suggest_int('hidden_dim1', 32, 128, step=16)
    hidden_dim2 = trial.suggest_int('hidden_dim2', 16, 64, step=16)
    latent_dim = trial.suggest_int('latent_dim', 2, 16, step=2)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_int('batch_size', 16, 64, step=16)
    
    hidden_dims = [hidden_dim1, hidden_dim2]
    
    # Split data into training and validation sets
    X_train, X_val = train_test_split(data_scaled, test_size=0.2, random_state=42)

    # Train the VAE using training and validation data
    vae, encoder, decoder = build_vae(input_dim=data_scaled.shape[1], hidden_dims=hidden_dims, latent_dim=latent_dim)
    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    history = vae.fit(X_train, X_train, validation_data=(X_val, X_val), epochs=100, batch_size=batch_size, verbose=0)
    
    # Access 'val_loss' after adding validation data
    val_loss = min(history.history['val_loss'])
    return val_loss

# Run Optuna study using TPE
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=100)

# Get the best hyperparameters from Optuna
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Train the final VAE using the best hyperparameters
hidden_dims = [best_params['hidden_dim1'], best_params['hidden_dim2']]
latent_dim = best_params['latent_dim']
learning_rate = best_params['learning_rate']
batch_size = best_params['batch_size']

vae, encoder, decoder = build_vae(input_dim=data_scaled.shape[1], hidden_dims=hidden_dims, latent_dim=latent_dim)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
history = vae.fit(data_scaled, data_scaled, epochs=100, batch_size=batch_size, verbose=1)

# Augment the normal data (generate 2000 normal samples)
desired_normal_size = 2000
augmented_normal_data_list = []
while len(augmented_normal_data_list) < desired_normal_size:
    new_samples = vae.predict(normal_data_scaled)
    augmented_normal_data_list.extend(new_samples)

augmented_normal_data = pd.DataFrame(augmented_normal_data_list[:desired_normal_size], columns=normal_data.columns[:-1])
augmented_normal_data['label_column'] = 0  # Label for normal data

# Augment the anomalies (generate 500 anomaly samples)
desired_anomaly_size = 500
augmented_anomaly_data_list = []
while len(augmented_anomaly_data_list) < desired_anomaly_size:
    new_samples = vae.predict(anomalies_scaled)
    augmented_anomaly_data_list.extend(new_samples)

augmented_anomaly_data = pd.DataFrame(augmented_anomaly_data_list[:desired_anomaly_size], columns=anomalies.columns[:-1])
augmented_anomaly_data['label_column'] = 1  # Label for anomalies

# Combine the augmented data and the original data
augmented_data = pd.concat([augmented_normal_data, augmented_anomaly_data], axis=0)

# Shuffle the augmented data
augmented_data = shuffle(augmented_data, random_state=42)

# Check final shape of the augmented dataset
print(f"Final augmented data shape: {augmented_data.shape}")

# Save the augmented data to an Excel file
output_path = '/Users/Hsiao-ping.ni/ASU Dropbox/Hsiao-Ping Ni/PhD/Data/4th paper- Chiller/Diffusion_Imbalanced_Augmented_Chiller_Data.xlsx'
#output_path = '/Users/hsiaopingni/Desktop/Diffusion_Imbalanced_Augmented_Chiller_Data.xlsx'
augmented_data.to_excel(output_path, index=False)
print(f"Augmented balanced data saved to {output_path}.")


[I 2024-11-25 09:18:25,151] A new study created in memory with name: no-name-0328ed9b-6911-45f4-aa6b-aa5b8dc40ca2
C:\Users\Hsiao-ping.ni\AppData\Local\Temp\ipykernel_56780\218452991.py:82: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
[I 2024-11-25 09:18:29,317] Trial 0 finished with value: 0.9697014689445496 and parameters: {'hidden_dim1': 48, 'hidden_dim2': 16, 'latent_dim': 14, 'learning_rate': 0.0006243751409959713, 'batch_size': 64}. Best is trial 0 with value: 0.9697014689445496.
[I 2024-11-25 09:18:33,169] Trial 1 finished with value: 0.9137906432151794 and parameters: {'hidden_dim1': 96, 'hidden_dim2': 64, 'latent_dim': 10, 'learning_rate': 0.004837007441645962, 'batch_size': 32}. Best is trial 1 with value: 0.9137906432151794.
[I 2024-11-25 09:18:36,

[I 2024-11-25 09:20:32,907] Trial 31 finished with value: 0.8925666809082031 and parameters: {'hidden_dim1': 112, 'hidden_dim2': 48, 'latent_dim': 8, 'learning_rate': 0.006939479756489666, 'batch_size': 16}. Best is trial 6 with value: 0.8031551837921143.
[I 2024-11-25 09:20:36,585] Trial 32 finished with value: 0.8972999453544617 and parameters: {'hidden_dim1': 128, 'hidden_dim2': 48, 'latent_dim': 6, 'learning_rate': 0.00817446267540806, 'batch_size': 16}. Best is trial 6 with value: 0.8031551837921143.
[I 2024-11-25 09:20:40,545] Trial 33 finished with value: 0.9035314917564392 and parameters: {'hidden_dim1': 96, 'hidden_dim2': 32, 'latent_dim': 10, 'learning_rate': 0.004456912072286645, 'batch_size': 16}. Best is trial 6 with value: 0.8031551837921143.
[I 2024-11-25 09:20:44,364] Trial 34 finished with value: 0.8741876482963562 and parameters: {'hidden_dim1': 96, 'hidden_dim2': 64, 'latent_dim': 10, 'learning_rate': 0.007669539770266366, 'batch_size': 16}. Best is trial 6 with valu

[I 2024-11-25 09:22:39,617] Trial 63 finished with value: 0.847309410572052 and parameters: {'hidden_dim1': 112, 'hidden_dim2': 48, 'latent_dim': 10, 'learning_rate': 0.0051471395831015595, 'batch_size': 16}. Best is trial 6 with value: 0.8031551837921143.
[I 2024-11-25 09:22:43,142] Trial 64 finished with value: 0.9081055521965027 and parameters: {'hidden_dim1': 112, 'hidden_dim2': 48, 'latent_dim': 8, 'learning_rate': 0.004110550919053191, 'batch_size': 16}. Best is trial 6 with value: 0.8031551837921143.
[I 2024-11-25 09:22:46,516] Trial 65 finished with value: 0.9396368861198425 and parameters: {'hidden_dim1': 112, 'hidden_dim2': 64, 'latent_dim': 12, 'learning_rate': 0.0030089197052295003, 'batch_size': 16}. Best is trial 6 with value: 0.8031551837921143.
[I 2024-11-25 09:22:49,955] Trial 66 finished with value: 0.9713025689125061 and parameters: {'hidden_dim1': 112, 'hidden_dim2': 32, 'latent_dim': 10, 'learning_rate': 0.005122340290406978, 'batch_size': 16}. Best is trial 6 with

[I 2024-11-25 09:24:36,130] Trial 95 finished with value: 0.9262568354606628 and parameters: {'hidden_dim1': 112, 'hidden_dim2': 32, 'latent_dim': 10, 'learning_rate': 0.005620652701484362, 'batch_size': 16}. Best is trial 76 with value: 0.6872785687446594.
[I 2024-11-25 09:24:40,205] Trial 96 finished with value: 0.923095703125 and parameters: {'hidden_dim1': 80, 'hidden_dim2': 48, 'latent_dim': 12, 'learning_rate': 0.006474232494160238, 'batch_size': 16}. Best is trial 76 with value: 0.6872785687446594.
[I 2024-11-25 09:24:43,847] Trial 97 finished with value: 0.9242623448371887 and parameters: {'hidden_dim1': 64, 'hidden_dim2': 32, 'latent_dim': 8, 'learning_rate': 0.00766530652218575, 'batch_size': 16}. Best is trial 76 with value: 0.6872785687446594.
[I 2024-11-25 09:24:47,527] Trial 98 finished with value: 0.9156362414360046 and parameters: {'hidden_dim1': 128, 'hidden_dim2': 48, 'latent_dim': 10, 'learning_rate': 0.0042037299431782384, 'batch_size': 16}. Best is trial 76 with va

Best hyperparameters: {'hidden_dim1': 128, 'hidden_dim2': 48, 'latent_dim': 10, 'learning_rate': 0.008866708901565701, 'batch_size': 16}
Epoch 1/100
4/4 [==============================] - 1s 2ms/step - loss: 1.1361
Epoch 2/100
4/4 [==============================] - 0s 2ms/step - loss: 0.9846
Epoch 3/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8906
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 0.9150
Epoch 5/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8904
Epoch 6/100
4/4 [==============================] - 0s 2ms/step - loss: 0.9082
Epoch 7/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8718
Epoch 8/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8803
Epoch 9/100
4/4 [==============================] - 0s 2ms/step - loss: 0.9740
Epoch 10/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8603
Epoch 11/100
4/4 [==============================] - 0s 2ms/step - loss: 0.9085
Epo

1/1 [==============================] - 0s 18ms/step
Final augmented data shape: (2500, 13)
Augmented balanced data saved to /Users/Hsiao-ping.ni/ASU Dropbox/Hsiao-Ping Ni/PhD/Data/4th paper- Chiller/Diffusion_Imbalanced_Augmented_Chiller_Data.xlsx.


In [6]:
from scipy.stats import ks_2samp

# Calculate mean and standard deviation for each parameter
original_stats = data.describe().loc[['mean', 'std']]
augmented_stats = augmented_data.describe().loc[['mean', 'std']]

In [7]:
original_stats

,ï»¿Discharge Temp (F),Input % full load amps (Motor) (%),Condenser liq temp IN (F),Condenser liq temp OUT (F),Chilled liq temp IN (F),Chilled liq temp OUT (F),Condenser saturation (F),Evaporator saturation (F),Evaporator pressure (PSIG),Condenser pressure (PSIG),Oil sump temp (F),Oil pressure (PSIG),Anomaly
mean,102.970000,50.900000,77.975000,83.655000,47.320000,40.513333,83.876667,39.725000,34.775000,93.911667,131.076667,35.027667,0.050000
std,6.529731,7.772594,7.268335,7.399116,1.284167,0.076947,7.777899,0.287376,0.272232,14.149553,1.876561,0.421637,0.219784


In [8]:
augmented_stats

,ï»¿Discharge Temp (F),Input % full load amps (Motor) (%),Condenser liq temp IN (F),Condenser liq temp OUT (F),Chilled liq temp IN (F),Chilled liq temp OUT (F),Condenser saturation (F),Evaporator saturation (F),Evaporator pressure (PSIG),Condenser pressure (PSIG),Oil sump temp (F),Oil pressure (PSIG),label_column
mean,0.287288,0.386616,0.315057,0.348286,0.683933,0.375610,0.376395,0.608584,0.618676,0.355833,0.549466,0.497163,0.20000
std,0.105049,0.028895,0.098183,0.096757,0.015667,0.014256,0.090175,0.024745,0.027693,0.093061,0.028772,0.010989,0.40008
